In [300]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy.optimize import minimize

In [301]:
#加载数据
df = pd.read_excel('各参数SEA.xlsx')
# print(df.sample(10))

In [302]:
#描述性统计
print(df.describe())
print(sea_range := df['SEA'].max() - df['SEA'].min())

             α(°)       β(°)       l1/l2        t_mm          SEA
count  560.000000  560.00000  560.000000  560.000000   560.000000
mean    45.000000   55.00000    2.250000    1.150000   669.679696
std     10.008941    7.07739    0.559517    0.111903   151.469724
min     30.000000   45.00000    1.500000    1.000000   368.340000
25%     35.000000   50.00000    1.875000    1.075000   544.565000
50%     45.000000   55.00000    2.250000    1.150000   684.160000
75%     55.000000   60.00000    2.625000    1.225000   785.387500
max     60.000000   65.00000    3.000000    1.300000  1023.070000
654.73


In [303]:
#数据拆分
X_train, X_test, y_train, y_test = train_test_split(
    np.array(df[['α(°)',  'β(°)',  'l1/l2',  't_mm',]]),
    np.array(df[['SEA']]),
    test_size=0.2,
    shuffle=True,
)

In [304]:
#训练多项式回归模型
model = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=4)),
    ('linear', RidgeCV(alphas=np.logspace(-3, 3, 6, base=10), cv=10),),
])
model.fit(X_train, y_train)

,steps,"[('scaler', ...), ('poly', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,degree,4
,interaction_only,False
,include_bias,True
,order,'C'


In [305]:
#模型评估
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2_test = r2_score(y_test, y_pred)
r2_train = r2_score(y_train, model.predict(X_train))
print(rmse)
print(r2_test)
print(r2_train)

20.196815092162492
0.9821713037716031
0.986471696372635


In [306]:
#优化
##定义目标函数
def objective(x):
    x = np.array(x).reshape(1, -1)
    y = model.predict(x)
    return -y

In [348]:
##定义边界约束
param_bounds = []
for i in df.iloc[:, 0:4]:
    param_bounds.append((df[i].min(), df[i].max()))
# print(param_bounds)

[(30, 60), (45, 65), (1.5, 3.0), (1.0, 1.3)]


In [347]:
times = 20
best_result = None
for i in range(times):
    if i < times:
        x0 = np.array([np.random.uniform(low, high) for low, high in param_bounds])
    else:
        x0 = np.array(df.iloc[:, 0:4].median())
    result = minimize(
                objective,
                x0,
                bounds=param_bounds,
                method='SLSQP',
                options=dict(
                    disp=False,
                    maxiter=100,
                    )
                )
    print(f'第{i+1}次起始点{x0}结果{-result.fun}')
    if best_result is None or result.fun < best_result.fun:
        best_result = result
optimal_params = best_result.x
optimal_sea = -best_result.fun
print(optimal_params)
print(optimal_sea)

第1次起始点[42.03080915 60.15007522  2.12291797  1.16000959]结果996.6890655594739
第2次起始点[46.93298199 63.18715685  2.7916777   1.20203521]结果996.6890655654364
第3次起始点[52.7585036  48.57029504  2.84333424  1.22393322]结果996.6890655524869
第4次起始点[58.79341546 48.79673291  2.53130054  1.2476139 ]结果996.6890654883355
第5次起始点[55.21166303 47.86315635  2.71992668  1.28225353]结果996.6890654089343
第6次起始点[58.86680448 52.37532584  2.09307227  1.11265773]结果996.6890655558158
第7次起始点[54.18515605 47.89907799  1.61404383  1.26243507]结果996.6890655654578
第8次起始点[46.27660761 59.38149165  2.46533535  1.07230315]结果996.6890655650725
第9次起始点[31.06649917 63.35061038  2.6767976   1.16749921]结果996.6890655635938
第10次起始点[59.74085764 55.95045071  1.98120683  1.26291438]结果996.6890655464483
第11次起始点[39.79968319 52.82020771  2.88115773  1.25972914]结果996.6890655646157
第12次起始点[34.14211355 53.24103417  1.79877094  1.29960688]结果996.6890654546394
第13次起始点[39.94483334 52.94576732  2.30978396  1.1176313 ]结果996.6890652655507
第14次起始点[30.39795191 6